# Interactive Visualization of Daytimes and Workingtimes in Europe

## Data

In [1]:
import pandas as pd
import geopandas as gpd

RELOAD_DATA = False
eu_data_path = 'datasets/saved/eu_gpd.geojson'
city_data_path = 'datasets/saved/city_data.csv'
sunset_data = 'datasets/saved/sunset_data_2022.geojson'
avg_country_data_path = 'datasets/saved/avg_country.csv'

### EU countries to capitals and GeoPandas data

In [2]:
from sun_data import get_sunset_sunrise_data
from geo_utils import load_eu_countries_as_geopandas, get_eu_city_data, get_avg_country_data

if RELOAD_DATA:
    print('Reloading data from datasets/saved folder ...')
    eu_gpd = gpd.read_file(eu_data_path)
    top_city_data = pd.read_csv(city_data_path)
    sun_data_gpd = gpd.read_file(sunset_data)
    avg_country_data = pd.read_csv(avg_country_data_path)
else:
    print('Generating data ...')
    top_city_data = get_eu_city_data(3)
    top_city_data.to_csv(city_data_path, index=False)

    eu_gpd = load_eu_countries_as_geopandas()
    eu_gpd.to_file(eu_data_path, driver='GeoJSON', index=False)

    avg_country_data = get_avg_country_data(top_city_data, eu_gpd)
    avg_country_data.to_csv(avg_country_data_path, index=False)

    sun_data_gpd = get_sunset_sunrise_data(year=2022, eu_gpd=eu_gpd, csv_path=sunset_data)

Generating data ...


C:\Users\phili\PycharmProjects\CircadianRythmEU\geo_utils.py:30: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
C:\Users\phili\PycharmProjects\CircadianRythmEU\geo_utils.py:30: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


Could not process day 2022-06-16 00:00:00


In [7]:
eu_gpd.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,iso_a2
21,5347896.0,Europe,Norway,NOR,403336,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...",NO
43,67059887.0,Europe,France,FRA,2715518,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3...",FR
110,10285453.0,Europe,Sweden,SWE,530883,"POLYGON ((11.02737 58.85615, 11.46827 59.43239...",SE
111,9466856.0,Europe,Belarus,BLR,63080,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",BY
112,44385155.0,Europe,Ukraine,UKR,153781,"POLYGON ((32.15944 52.06125, 32.41206 52.28869...",UA


In [4]:
top_city_data.head()

,population,CODE,country_ISO_A2,NAME,longitude,latitude,mercantor_x,mercantor_y,social_timezone,utc_sun_timezone_offset,longitudinal_diff,longitudinal_diff_km
0,1205492,BE001C,BE,Bruxelles/Brussel,4.351697,50.846557,4.844287e+05,6.594196e+06,Europe/Brussels,1.0,10.648303,851.864240
1,523591,BE002C,BE,Antwerpen,4.399708,51.221110,4.897733e+05,6.660499e+06,Europe/Brussels,1.0,10.600292,848.023352
2,383710,BE005C,BE,Liège,5.573611,50.645094,6.204516e+05,6.558754e+06,Europe/Brussels,1.0,9.426389,754.111104
3,1238438,BG001C,BG,Sofia,23.321736,42.697703,2.596164e+06,5.266072e+06,Europe/Sofia,2.0,6.678264,534.261128
4,345213,BG002C,BG,Plovdiv,24.749930,42.141854,2.755150e+06,5.182252e+06,Europe/Sofia,2.0,5.250070,420.005624


In [5]:
sun_data_gpd.head()

,iso_a3,capital,day,month,year,sunrise_UTC,sunset_UTC,geometry
0,ALB,Tirana,1,1,2022,6,15,"POLYGON ((2339940.185 4989171.529, 2337708.178..."
1,ALB,Tirana,2,1,2022,6,15,"POLYGON ((2339940.185 4989171.529, 2337708.178..."
2,ALB,Tirana,3,1,2022,6,15,"POLYGON ((2339940.185 4989171.529, 2337708.178..."
3,ALB,Tirana,4,1,2022,6,15,"POLYGON ((2339940.185 4989171.529, 2337708.178..."
4,ALB,Tirana,5,1,2022,6,15,"POLYGON ((2339940.185 4989171.529, 2337708.178..."


### Generate top n cities per country with timezone features

## Interactive visualization with Panel/Bokeh

In [49]:
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, ColumnDataSource, LabelSet, HoverTool
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Plasma256
from bokeh.palettes import brewer
import json
import panel as pn
from datetime import datetime, date
import numpy as np

data_field = 'sunrise_UTC'
bokeh_tools = 'wheel_zoom, pan, box_zoom, reset'
colorbar_settings = {'title_text_font_size':'12pt','label_standoff':12}

def get_bokeh_geodata_source(gpd_df):
    json_data = json.dumps(json.loads(gpd_df.to_json()))
    return GeoJSONDataSource(geojson = json_data)

def bokeh_plot_map(data):
    p = figure(toolbar_location='right', tools=bokeh_tools, active_scroll ="wheel_zoom",
               title="Distance to eastern timezone meridian for large EU cities",
               x_range=(top_city_data['mercantor_x'].min(),top_city_data['mercantor_x'].max()))
    p.title.text_font_size = '20px'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    # AD MAP TILES ---------------------------------------------------------------------------
    p.add_tile(Vendors.CARTODBPOSITRON_RETINA)

    # ADD GEO STUFF FOR COUNTRIES AS A WHOLE -------------------------------------------------
    geo_data_source = get_bokeh_geodata_source(data)

    values = data[data_field]
    palette = brewer['OrRd'][8]
    palette = palette[::-1]
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = values.min(), high = values.max())
    color_bar = ColorBar(color_mapper=color_mapper, location=(0,0), title='Sunrise (UTC)', **colorbar_settings)
    country_suntimes = p.patches('xs','ys', source=geo_data_source,
            fill_color={'field': data_field, 'transform': color_mapper},
            line_color='blue',
            line_width=0.5,
            fill_alpha=0.8)
    p.add_layout(color_bar, 'below')

      # TOOLTIPS FOR COUNTRY PATCHES
    tooltips_country = [
        ('Country', '@iso_a3'),
        ('Sunrise (UTC)', '@sunrise_UTC'),
        ('Sunset (UTC)', '@sunset_UTC')
    ]
    p.add_tools(HoverTool(renderers=[country_suntimes], tooltips=tooltips_country))

    # ===================================================================================================================

    #ADD BARS FOR DISTANCE TO EAST MERIDIAN EFFECT
    length_scale = 200000
    bar_data_source = ColumnDataSource(dict(
            x0=avg_country_data['mercantor_x'],
            y0=avg_country_data['mercantor_y'],
            x1=avg_country_data['mercantor_x'] + (length_scale *  avg_country_data['norm_weighted_mean_longdiff']) * np.sign(avg_country_data['weighted_mean_longdiff']),
            y1=avg_country_data['mercantor_y']
        )
    )
    divider_len = 50000
    divider_data_source = ColumnDataSource(dict(
            x0=avg_country_data['mercantor_x'],
            y0=avg_country_data['mercantor_y'] - divider_len / 2,
            x1=avg_country_data['mercantor_x'],
            y1=avg_country_data['mercantor_y'] + divider_len / 2,
            text=avg_country_data['name'],
            text_y=(avg_country_data['mercantor_y'] + divider_len / 2) + 1000
    ))
    longdiff_quads = p.segment(x0="x0", y0="y0", x1="x1", y1="y1", line_width=5, source=bar_data_source)
    londiff_diviers = p.segment(x0="x0", y0="y0", x1="x1", y1="y1", line_width=2, source=divider_data_source)
    longdiff_label = p.text(x="x0",y="text_y", text="text", source=divider_data_source)

    # TOOLTIPS FOR CITY DATA
    tooltips_city = [
        ('Country', '@iso_a2'),
        ('City', '@NAME'),
        ('Relative position (dist) to timezone border', '@longitudinal_diff_km')
    ]
    p.add_tools(HoverTool(renderers=[longdiff_quads, londiff_diviers], tooltips=tooltips_city, name='test'))
    return p

In [4]:
from bokeh.models import DataTable, TableColumn

def bokeh_country_table(country_data):
    country_data_sorted = country_data.sort_values('weighted_mean_longdiff', ascending=False)
    source = ColumnDataSource(country_data_sorted)
    columns = [
        TableColumn(field='name', title='Country Name'),
        TableColumn(field="iso_a2",title="Country Code (ISO_A2)"),
        TableColumn(field="social_timezone", title="Social Timezone"),
        TableColumn(field="weighted_mean_longdiff", title="Weighted (pop.size) avg. dist. to east meridian (km)"),
        TableColumn(field="pop_est", title="Estimated population")
    ]
    data_table = DataTable(source=source, columns=columns)
    return data_table

In [5]:
def bokeh_sun_table(sun_data):
    source = ColumnDataSource(sun_data)

    # Add data table
    columns = [
        TableColumn(field="iso_a3", title="Country Code (ISO_A3)"),
        TableColumn(field="year", title="Year"),
        TableColumn(field="month", title="Month"),
        TableColumn(field="day", title="Day"),
        TableColumn(field="sunrise_UTC", title="Sunrise (UTC/GMT)"),
        TableColumn(field="sunset_UTC", title="Sunrise (UTC/GMT)")
    ]
    data_table = DataTable(source=source, columns=columns)
    return data_table

In [50]:
def map_visualization():
    # CREATE MAP  ----------------------------------------------------------------------------------
    # Create Map Panel
    map_pane = pn.pane.Bokeh(sizing_mode='scale_both', width_policy='max')
    start_date = date(2022,1,1)
    end_date = date(2022,12,31)
    selected_date = pn.widgets.DateSlider(name='Date Slider', value=start_date, start=start_date, end=end_date)
    def update_map(event):
        d = selected_date.value
        selected_sundata = sun_data_gpd.query(f'day == {d.day} & month == {d.month} & year == {d.year}')
        map_pane.object = bokeh_plot_map(selected_sundata)
    selected_date.param.watch(update_map, 'value')
    selected_date.param.trigger('value')

    # CREATE DATATABLES ----------------------------------------------------------------------------------
    sizing_dict = dict(sizing_mode='stretch_both', width_policy='auto', margin=10)
    # Create City Table Panel
    country_data_pane = pn.pane.Bokeh(**sizing_dict)
    country_data_pane.object = bokeh_country_table(avg_country_data)

    # Create Sun Table Panel
    sun_data_pane = pn.pane.Bokeh(**sizing_dict)
    sun_data_pane.object = bokeh_sun_table(sun_data_gpd.iloc[:,:-1])

    # Create panel application layout
    map_vis = pn.Column(selected_date, map_pane)
    tabs = pn.Tabs(('Map', map_vis), ('Country Data', country_data_pane), ('Sun Data', sun_data_pane))
    return tabs

app = map_visualization()

In [51]:
app.show()

Launching server at http://localhost:2341
